In [1]:
from paddleocr import PaddleOCR,draw_ocr
import os
from PIL import Image
import torch
import pyiqa
from fuzzysearch import find_near_matches
import time

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `french`, `german`, `korean`, `japan`
# to switch the language model in order.
OCR=PaddleOCR(det_model_dir='C:/Users/aurelien.martinez/Documents/Basegun/3.OCR/PaddleModels/detection', rec_model_dir='C:/Users/aurelien.martinez/Documents/Basegun/3.OCR/PaddleModels/recognition', cls_model_dir='C:/Users/aurelien.martinez/Documents/Basegun/3.OCR/PaddleModels/classification', use_angle_cls=True,show_log = False) # need to run only once to download and load model into memory
images=os.listdir("./ImagesIRC/")

# inference on images

In [133]:
def get_text(results):
    """extracts raw text from PaddleOCR output
    Args:
        results: raw result from PaddleOCR

    Returns:
        text: A string with the text extracted from the image
    """
    text=" "
    for result in results:
        text=text+result[1][0]+" "
    return text.lower()


def is_in(word,phrase):
    """Check if a word is in a word using fuzzysearch algorithm for a tolerance error
    Args:
        word: word seek in the text
        phrase: text to explore

    Returns:
        boolean: true if word is in phrase
    """
    res=find_near_matches(word, phrase, max_l_dist=1)
    return len(res)>0
    
def is_alarm_model(text):
    """determine if the text is from an alarm model weapon image using rules defined with weapon experts
    Args:
        text: string of the extract text

    Returns:
        boolean: true if the an alarm model is recognized
    """
    #fuzzy search for words but exat value for model number
    zoraki=["r2", "925","92s","906","2906","918","9o6","29o6"]
    
    #Blow
    if is_in("blow",text):
        if any(word in text for word in ["f92","c75"]):
            return True
        else:
            return False
    #Zoraki
    elif is_in("zoraki",text):
        if any(word in text for word in zoraki):
            return True
        else:
            return False
    
    elif is_in("kimar",text):
        if is_in("auto",text):
            if "75" in text:
                return True
            else:
                return False
        elif "911" in text:
            return True
        else:
            return False
    elif is_in("auto",text):
        if "92" in text:
            return True
        else:
            return False
    elif is_in("alarm",text): #Sur ce type de modèle il arrive que le mot kimar soit remplacé par le logo
            if any(is_in(word,text) for word in ["competitive","power"]):
                return True
            else:
                return False
    
    else:
        return False

def is_pak(text):
    """determine if the text is from an alarm model weapon image with a PAK engraving
    Args:
        text: string of the extract text

    Returns:
        boolean: true if the PAK engraving is recognized
    """
    if any(word in text for word in ["pak ","p.a.k","pak."," pak","pa.k","p.ak"]):
        return True
    else:
        return False
    
def quality_eval(img):
    """Evaluate the CNNIQA for image quality and compare it to a defined threshold
    Args:
        img: PIL image

    Returns:
        boolean: true if the image has a good quality (score<threshold)
    """
    device = torch.device("cpu")
    metric = pyiqa.create_metric('cnniqa', device=device)
    res=metric(img)
    print(res)
    return res>QUALITY_THRESHOLD


def is_alarm_weapon(image_bytes):
    """Global pipeline for determining if the weapon is an alarm gun using OCR
    Args:
        image_bytes: Bytes image from Basegun

    Returns:
        string: User feedback on image quality or on alarm gun assessment
    """
    img = Image.open(io.BytesIO(image_bytes))

    if quality_eval(img):
        results = model_ocr.ocr(np.asarray(img), cls=True)
        print(results)
        if results!=[None]: #The results with recongition and detection confidence below 0.5 are filtered by paddle, the thresholds values can be changed
            text=get_text(results)
            if is_alarm_model(text):
                return "alarm weapon from model"
            elif is_pak(text):
                return "alarm weapon PAK"
            else:
                return "Not an alarm weapon"
        else:
            return "Text not detected please get closer to the weapon"
    else:
        return "The photo does not seem to have a good quality please take another photo"

In [ ]:
import numpy as np
dict={}
device = torch.device("cpu")
metric = pyiqa.create_metric('cnniqa', device=device)
for image in images:
    img = Image.open("./ImagesIRC/"+image)
    width, height = img.size
    ratio = 640/width
    newsize = (640, int(height*ratio))
    newsize2 = (1280, 2*int(height*ratio))
    im1 = img.resize(newsize)
    im2=img.resize(newsize2)
    a=time.time()
    quality= metric(im1)
    result = OCR.ocr("./ImagesIRC/"+image, cls=True)
    if result[0]==None:
        raw_text=""
        PAK = False
        alarm_model=False
    else:
        raw_text=get_text(result[0])
        PAK = is_pak(raw_text)
        alarm_model=is_alarm_model(raw_text)
    print(time.time()-a)
    dict[image]={"quality":quality,"result":result,"text":raw_text,"PAK_pred":PAK,"AlarmModel_pred":alarm_model}

# Preprocessing labelstudio file

In [135]:
import pandas as pd
import numpy as np

In [136]:
df=pd.read_csv('labels.csv')
df.replace(np.nan, '', inplace=True)
df['PAK'] = df['Arme Alarme'].apply(lambda x: 'PAK' in x)
df['AlarmModel'] = df['Arme Alarme'].apply(lambda x: 'Arme' in x)
df['VueGlobale'] = df['Analyse image'].apply(lambda x: 'Globale' in x)
df['Lisible'] = df['Analyse image'].apply(lambda x: 'Lisible' in x)

l = ['Lisible,','Lisible','Vue Globale,','Vue Globale']


df['TypeMarquage']=df['Analyse image'].apply(lambda x: x.split( ','))

df['TypeMarquage'].apply(lambda x: x.remove('Lisible') if 'Lisible' in x else x)

df['TypeMarquage'].apply(lambda x: x.remove('Vue Globale') if 'Vue Globale' in x else x)

df.drop(columns=['Arme Alarme', 'info','Analyse image'],inplace=True)
df.set_index('image', inplace=True)


# Comparaison des resultats

In [137]:
labels=df

In [138]:
pred= pd.DataFrame(dict)
pred=pred.T

In [139]:
comparison=labels.merge(pred,left_index=True, right_index=True)

In [ ]:
comparison

## Metrics on all dataset

In [ ]:
def precisionPAK(df):
    return len(df[(df['PAK']==df['PAK_pred']) & (df['PAK']==True)])/len(df[(df['PAK_pred']==True)])

def precisionModel(df):
    return len(df[(df['AlarmModel']==df['AlarmModel_pred']) & (df['AlarmModel']==True)])/len(df[(df['AlarmModel_pred']==True)])

print(precisionPAK(comparison))
print(precisionModel(comparison))

In [142]:
def value_graphPAK(df):
    print(len(df[(df['PAK']==True)]))
    print(len(df[(df['PAK']==df['PAK_pred']) & (df['PAK']==True)]))
    print(len(df[(df['PAK']==False)]))
    print(len(df[(df['PAK_pred']==True) & (df['PAK']==False)]))

def value_graphModel(df):
    print(len(df[(df['AlarmModel']==True)]))
    print(len(df[(df['AlarmModel']==df['AlarmModel_pred']) & (df['AlarmModel']==True)]))
    print(len(df[(df['AlarmModel']==False)]))
    print(len(df[(df['AlarmModel_pred']==True) & (df['AlarmModel']==False)]))

In [ ]:
value_graphModel(comparison[(comparison['VueGlobale']==False)])
value_graphPAK(comparison[(comparison['VueGlobale']==False)])

In [ ]:
def recallPAK(df):
    return len(df[(df['PAK']==df['PAK_pred']) & (df['PAK']==True)])/len(df[(df['PAK']==True)])

def recallModel(df):
    return len(df[(df['AlarmModel']==df['AlarmModel_pred']) & (df['AlarmModel']==True)])/len(df[(df['AlarmModel']==True)])

print(recallPAK((comparison)))
print(recallModel(comparison))
len(comparison[(comparison['VueGlobale']==False)])

## Metrics on images with enough quality

In [ ]:
comparisonQuality=comparison[(comparison['quality']>0.50)]
print(len(comparisonQuality))

print(precisionPAK(comparisonQuality))
print(precisionModel(comparisonQuality))

print(recallPAK(comparisonQuality))
print(recallModel(comparisonQuality))

In [ ]:
value_graphModel(comparisonQuality)
value_graphPAK(comparisonQuality)

## Metrics on images with enough quality and lisible

In [ ]:
comparisonQualityLisible=comparison[(comparison['quality']>0.50) & (comparison['Lisible'])]
print(len(comparisonQualityLisible))

print(precisionPAK(comparisonQualityLisible))
print(precisionModel(comparisonQualityLisible))

print(recallPAK(comparisonQualityLisible))
print(recallModel(comparisonQualityLisible))

In [ ]:
value_graphModel(comparisonQualityLisible[comparisonQualityLisible['VueGlobale']==False])
value_graphPAK(comparisonQualityLisible[comparisonQualityLisible['VueGlobale']==False])

## Metrics on images with enough quality and lisible et vue rapprochée

In [ ]:
comparisonQualityLisible=comparison[(comparison['quality']>0.50) & (comparison['Lisible'])&(comparison['VueGlobale']==False)]
print(len(comparisonQualityLisible))

print(precisionPAK(comparisonQualityLisible))
print(precisionModel(comparisonQualityLisible))

print(recallPAK(comparisonQualityLisible))
print(recallModel(comparisonQualityLisible))

## Metrics on images with enough quality and lisible et vue rapprochée par type de marquage

In [ ]:
comparisonQualityLisible2=comparison[(comparison['quality']>0.50) & (comparison['Lisible'])&(comparison['VueGlobale']==False)]

comparisonQualityLisible2['TypeMarquage2']=comparisonQualityLisible2['TypeMarquage'].apply(lambda x: x[0] if len(x)>0 else '')
comparisonQualityLisible2=comparisonQualityLisible2[comparisonQualityLisible2["TypeMarquage2"]=="Gravure"]
print(len(comparisonQualityLisible2))

print(precisionPAK(comparisonQualityLisible2))
print(precisionModel(comparisonQualityLisible2))

print(recallPAK(comparisonQualityLisible2))
print(recallModel(comparisonQualityLisible2))

# Identification du type d'erreurs

In [82]:
PAK_not_detected=comparisonQualityLisible[(comparisonQualityLisible['PAK']!=comparisonQualityLisible['PAK_pred']) & (comparisonQualityLisible['PAK']==True)&(comparison['VueGlobale']==False)]

In [83]:
images=list(PAK_not_detected.index)
labels=list(PAK_not_detected['text'])
quality=list(PAK_not_detected['quality'])
i=0

In [ ]:
len(images)

In [ ]:

img = Image.open("./ImagesIRC/"+images[i])
print(labels[i],quality[i])
display(img)
i=i+1

In [363]:
Model_not_detected=comparisonQualityLisible[(comparisonQualityLisible['AlarmModel_pred']==False) & (comparisonQualityLisible['AlarmModel']==True)]

In [364]:
images=list(Model_not_detected.index)
labels=list(Model_not_detected['text'])
quality=list(Model_not_detected['quality'])
i=0

In [ ]:
len(images)

In [ ]:

img = Image.open("./ImagesIRC/"+images[i])
print(labels[i],quality[i])
display(img)
i=i+1

# Statistiques

In [297]:
df=labels
df['TypeMarquage2']=df['TypeMarquage'].apply(lambda x: x[0] if len(x)>0 else '')
df=df.groupby('TypeMarquage2').size()


In [ ]:
df